In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from scipy import signal
from scipy import stats
from scipy import interpolate
import matplotlib.pyplot as plt
import cv2
import os

In [5]:
def area_of_triangle(A,B,C):
    """
    computes the area of a triangle given by 3 points in 2d or 3d coordinates
    A, B and C must be numpy arrays
    it works with vectors A=[a1,a2,a3], B=[b1,b2,b3], c=[c1,c2,c3]
    or matrices
    A=[[a11,a12,a13],...[a1n,a2n,b3n]], B=[[b11,b12,b13],...[b1n,b2n,b3n]], C=[[c11,c12,c13],...[c1n,c2n,c3n]]
    """
    
    As = A.shape
    Bs = B.shape
    Cs = C.shape
    
    if len(As) == 1 : 
        #we got vectors
        if (As[1]>3) or (Bs[1]>3) or (Cs[1]>3):
            raise Exception('coordinates can only be 2d or 3d')
            return None
    else:
        #check at least one of the dimensions is two or three
        if (As[0]==2) or (As[0]==3) or (As[1]==2) or (As[1]==3):
            #one of the dimensions of A is 2 or 3, now check that all the vectors have the same size
            if (As!=Bs) or (As!=Cs):
                raise Exception('vectors must be the same size')
                return None
            else:
                #move forward
                pass
        else:
            raise Exception('coordinates can only be 2d or 3d')
            return None
        
    
    #at this point we know that one of the dimensions has 2 or 3 elements we move  forward assuming that 
    #the user provided the vectors with the correct size

    #move all vectors to the same origin -> A
    AB = B-A
    AC = C-A
    
    if len(As) == 1 :
        if As[1]==3:
            #if only one vector then simply compute the norm of the cross product
            area = (1/2)*np.linalg.norm(np.cross(AB,AC))
        else:
            AB = np.column_stack((AB,np.zeros((As[0],1))))
            AC = np.column_stack((AC,np.zeros((As[0],1))))
            area = (1/2)*np.linalg.norm(np.cross(AB,AC)) 
    else:
        if As[1]==3:
            #if  multiple vectors compute the norm along the axis one 
            area = (1/2)*np.linalg.norm(np.cross(AB,AC), axis = 1)  
        else:
            AB = np.column_stack((AB,np.zeros((As[0],1))))
            AC = np.column_stack((AC,np.zeros((As[0],1))))
            area = (1/2)*np.linalg.norm(np.cross(AB,AC), axis = 1)     
    
    return area

def three_point_difference(x,time=None):
    #computes three point difference for derivative 
    
    if time is None:
        h = 1
    else:
        h = np.mean(np.diff(time))
        
    
    dx = np.zeros(x.shape)
    dx[0] = (1/(2*h))*(-3*x[0]+4*x[1]-x[2])
    dx[1] = (1/(2*h))*(-x[0]+x[2])
    dx[2:]= (1/(2*h))*(x[0:-2]-4*x[1:-1]+3*x[2:])
    
    return dx

def adjust_amplitude_and_time(sig, time,normalize=True, des_n = None):
    """
    Takes a signal x of lengt n and return a new signal x_n of lenght des_n and with zero meand and unit standard deviation
    
    Interpolation is performed using cubic splines
    """
    sig = sig[:,None]
    time = time [:,None]
    
    if des_n is None:
        des_time = time
    else:
        des_time = np.linspace(time[0],time[-1],des_n)
 
    if normalize:
        sig = (sig-np.mean(sig))/np.std(sig)
    try:
        tck = interpolate.splrep(time, sig, s=0)
    except:
        tck = interpolate.splrep(np.sort(time,axis=None), sig, s=0)
    
    new_sig = interpolate.splev(des_time, tck, der=0)
    
    new_sig_der = interpolate.splev(des_time, tck, der=1)
    
    return new_sig, des_time, new_sig_der 
    
def compute_excentricity(W,O):
    """
    Compute the excentricity of a ellipsis with wide W and opening  O
    """
    e = np.zeros(W.shape)
    for k,n in enumerate(zip(W,O)):
        w,o = abs(n[0]),abs(n[1])
        if w<o:
            e[k] = np.sqrt(1-(w/o)**2)
        elif w>o:
            e[k] = np.sqrt(1-(o/w)**2)
        else: 
            e[k] = 1
    
    return e
    
def concordance_correlation_coefficient(s1, s2):
    N1 = len(s1)
    N2 = len(s2)
    if N1==N2:
        N = N1
    elif N1>N2:
        s1 = s1[0:N2]
        N = N2
    elif N1<N2:
        s2 = s2[0:N1]
        N = N1
        
    m_s1 = np.mean(s1)
    m_s2 = np.mean(s2)
    
    s1_nomean = s1-m_s1
    s2_nomean = s2-m_s2
    
    s1_ss2 = (1/N)*np.sum((s1_nomean)**2)
    s2_ss2 = (1/N)*np.sum((s2_nomean)**2)
       
    s1s2 = (1/N)*np.sum(np.multiply(s1_nomean,s2_nomean)) # np.multiply(A,B) -> element wise multiplication between matrices A and B
    
    p = (2*s1s2)/(s1_ss2 + s2_ss2 +(m_s1 - m_s2)**2)
    
    return p


def get_mouth_positions_3d(DF,coord='3d'):

    #DF_3dpositions is a DataFrame

    #we are goinf to focus on the rihgt and left corners of the mouth and the top and bottom lips 
    #these positions correspond to landmarks 48, 54 and 51, 57 respectivelly
    if coord is '3d':
        DF_3dpositions = DF
        Right_corner = DF_3dpositions.filter(like='landmark_48')
        Right_corner_x = Right_corner.iloc[1:,[0]].values
        Right_corner_y = Right_corner.iloc[1:,[1]].values
        Right_corner_z = Right_corner.iloc[1:,[2]].values
        Right_Corner_Coord = np.column_stack((Right_corner_x.astype(np.double),Right_corner_y.astype(np.double),Right_corner_z.astype(np.double)))

        Left_corner = DF_3dpositions.filter(like='landmark_54')
        Left_corner_x = Left_corner.iloc[1:,[0]].values
        Left_corner_y = Left_corner.iloc[1:,[1]].values
        Left_corner_z = Left_corner.iloc[1:,[2]].values
        Left_Corner_Coord = np.column_stack((Left_corner_x.astype(np.double),Left_corner_y.astype(np.double),Left_corner_z.astype(np.double)))

        Top_lip = DF_3dpositions.filter(like='landmark_51')
        Top_lip_x = Top_lip.iloc[1:,[0]].values
        Top_lip_y = Top_lip.iloc[1:,[1]].values
        Top_lip_z = Top_lip.iloc[1:,[2]].values
        Top_Lip_Coord = np.column_stack((Top_lip_x.astype(np.double),Top_lip_y.astype(np.double),Top_lip_z.astype(np.double)))

        Bottom_lip = DF_3dpositions.filter(like='landmark_57')
        Bottom_lip_x = Bottom_lip.iloc[1:,[0]].values
        Bottom_lip_y = Bottom_lip.iloc[1:,[1]].values
        Bottom_lip_z = Bottom_lip.iloc[1:,[2]].values
        Bottom_Lip_Coord = np.column_stack((Bottom_lip_x.astype(np.double),Bottom_lip_y.astype(np.double),Bottom_lip_z.astype(np.double)))

        Nose_tip= DF_3dpositions.filter(like='landmark_30')
        Nose_tip_x = Nose_tip.iloc[1:,[0]].values
        Nose_tip_y = Nose_tip.iloc[1:,[1]].values
        Nose_tip_z = Nose_tip.iloc[1:,[2]].values
        Nose_Tip_Coord = np.column_stack((Nose_tip_x.astype(np.double),Nose_tip_y.astype(np.double),Nose_tip_z.astype(np.double)))
        
        norm_factor = 1

    else:
        #DF_3dpositions is a DataFrame

        #we are goinf to focus on the rihgt and left corners of the mouth and the top and bottom lips 
        #these positions correspond to landmarks 48, 54 and 51, 57 respectivelly
        
        DF_2dpositions = DF
        
        Right_corner_x = DF_2dpositions.filter(like='landmark_48_x').values
        Right_corner_y = DF_2dpositions.filter(like='landmark_48_y').values
        Right_Corner_Coord = np.column_stack((Right_corner_x.astype(np.double),Right_corner_y.astype(np.double)))

        Left_corner = DF_2dpositions.filter(like='landmark_54')
        Left_corner_x = DF_2dpositions.filter(like='landmark_54_x').values
        Left_corner_y = DF_2dpositions.filter(like='landmark_54_y').values
        Left_Corner_Coord = np.column_stack((Left_corner_x.astype(np.double),Left_corner_y.astype(np.double)))

        Top_lip_x = DF_2dpositions.filter(like='landmark_51_x').values
        Top_lip_y = DF_2dpositions.filter(like='landmark_51_y').values
        Top_Lip_Coord = np.column_stack((Top_lip_x.astype(np.double),Top_lip_y.astype(np.double)))

        Bottom_lip_x = DF_2dpositions.filter(like='landmark_57_x').values
        Bottom_lip_y = DF_2dpositions.filter(like='landmark_57_y').values
        Bottom_Lip_Coord = np.column_stack((Bottom_lip_x.astype(np.double),Bottom_lip_y.astype(np.double)))

        Left_cantil_x = DF_2dpositions.filter(like='landmark_42_x').values
        Left_cantil_y = DF_2dpositions.filter(like='landmark_42_y').values
        Left_cantil_Coord = np.column_stack((Left_cantil_x.astype(np.double),Left_cantil_y.astype(np.double)))


        Right_cantil_x = DF_2dpositions.filter(like='landmark_39_x').values
        Right_cantil_y = DF_2dpositions.filter(like='landmark_39_y').values
        Right_cantil_Coord = np.column_stack((Right_cantil_x.astype(np.double),Right_cantil_y.astype(np.double)))
        norm_factor = np.linalg.norm(Right_cantil_Coord - Left_cantil_Coord, axis=1)

        Nose_tip_x = DF_2dpositions.filter(like='landmark_30_x').values
        Nose_tip_y = DF_2dpositions.filter(like='landmark_30_y').values
        Nose_Tip_Coord = np.column_stack((Nose_tip_x.astype(np.double),Nose_tip_y.astype(np.double)))

        
    
                                         
                                         
    return Right_Corner_Coord, Left_Corner_Coord, Top_Lip_Coord, Bottom_Lip_Coord, Nose_Tip_Coord, np.mean(norm_factor)
    #return Right_Corner_Coord, Left_Corner_Coord, Top_Lip_Coord, Bottom_Lip_Coord, Nose_Tip_Coord

#     mouth_corners = np.zeros((len(Right_Corner_Coord),1))
#     top_bottom = np.zeros((len(Right_Corner_Coord),1))
#     for i in range(len(Right_Corner_Coord)):
#         mouth_corners[i] = np.sqrt((Right_Corner_Coord[i,0]-Left_Corner_Coord[i,0])**2+(Right_Corner_Coord[i,1]-Left_Corner_Coord[i,1])**2+(Right_Corner_Coord[i,2]-Left_Corner_Coord[i,2])**2)
#         top_bottom[i] = np.sqrt((Top_Lip_Coord[i,0]-Bottom_Lip_Coord[i,0])**2+(Top_Lip_Coord[i,1]-Bottom_Lip_Coord[i,1])**2+(Top_Lip_Coord[i,2]-Bottom_Lip_Coord[i,2])**2)


In [6]:
# get all files in path and process each one. Show progress with a bar
paths = [r'C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Healthy', r'C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Parkinsons']
#r'C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\example'

tasks = ['REST', 'PA', 'BBP', 'BIGSMILE']

# Features = Variables_placeholder()


# columnsDF = [ 
#             'Subject_ID', 
#             'Subject_status', 
#             'A_mean_rest', 
#             'A_right_mean_rest', 
#             'A_left_mean_rest', 
#             'TB_mean_REST',         
#             'Duration_var_PA', 
#             'Max_TB_PA',
#             'Max_TB_vel_PA', 
#             'Min_TB_vel_PA', 
#             'TB_path_cmd_BBP',
#             'Max_TB_BBP',
#             'Max_TB_vel_BBP', 
#             'Min_TB_vel_BBP', 
#             'A_mean_BBP', 
#             'Delta_A_BBP', 
#             'CCC_A_left_A_right_BBP', 
#             'A_mean_BIGSMILE',
#             'Delta_A_BIGSMILE', 
#             'CCC_A_left_A_right_BIGSMILE'
#             ]

columnsDF = [ 
            'Subject_ID', 
            'A_mean_rest', 
            'A_right_mean_rest', 
            'A_left_mean_rest', 
            'TB_mean_REST',         
            'LL_mean_REST', 
            'WM_mean_REST',
            ]

DataFrameResults = pd.DataFrame(columns=columnsDF)

coor_sys = '2d'

#get REST values first as they will be used to normalize additional values
for path in paths:
    Files = os.listdir(path)            
    if coor_sys == '3d':
        
        ext='Landmarks3D.csv'
    else:
        ext='landmarksFiltered.csv'
    Files = [i for i in Files if ext in i]
    for k,f in enumerate(Files):
        Files[k] = os.path.join(path,f)
        
#         Features = Variables_Placeholder()
        
        Subject_ID = f[0:4]  #get subject ID from file name
#         Features.Subject_status = f[0:2] #get disease status from file name
        
        this_task = None
        
        for t in tasks: 
            if t in f: this_task = t  #get task from file name
              
        DataFrame3dInfo = pd.read_csv(Files[k])
        Right_Corner_Coord, Left_Corner_Coord, Top_Lip_Coord, Bottom_Lip_Coord, Nose_Tip_Coord,_ = get_mouth_positions_3d(DataFrame3dInfo,coor_sys)  # get the four corners of the mouth :) 
        time_sec = DataFrame3dInfo['Time_Stamp (s)'].values[1:]

        datum = []
        #operations at rest
#         try:
        if this_task is 'REST':
            # i will consider only the middle 20% of the video to take the measurements 
            # this is done with the assumption that the subject was at 'rest' at least during the middle of the video 
            mid_point = len(Right_Corner_Coord)//2
            init_point = mid_point-int(len(Right_Corner_Coord)*0.15)
            end_point = mid_point+int(len(Right_Corner_Coord)*0.15)

            A_left = np.zeros((end_point-init_point))
            A_right = np.zeros((end_point-init_point))
            A = np.zeros((end_point-init_point))
            TB = np.zeros((end_point-init_point))

            A_l = area_of_triangle(Left_Corner_Coord[:,:], Top_Lip_Coord[:,:], Bottom_Lip_Coord[:,:])
            A_r = area_of_triangle(Right_Corner_Coord[:,:], Top_Lip_Coord[:,:], Bottom_Lip_Coord[:,:])
            A = A_l  + A_r

            TB = np.linalg.norm(Top_Lip_Coord - Bottom_Lip_Coord, axis =1)
            LL = np.linalg.norm(Nose_Tip_Coord - Bottom_Lip_Coord, axis =1)
            WM = np.linalg.norm(Right_Corner_Coord - Left_Corner_Coord, axis =1)

            fs = int(1/(time_sec[1]-time_sec[0]))
            nyq = 0.5 * fs
            highcut = 10
            order = 4
            high = highcut / nyq
            b, a = signal.butter(order,  high, btype='low')

            TB  = signal.filtfilt(b, a, TB)
            LL  = signal.filtfilt(b, a, LL)
            WM  = signal.filtfilt(b, a, WM)
            A_l =  signal.filtfilt(b, a, A_l)
            A_r = signal.filtfilt(b, a, A_r)
            A = A_l  + A_r

#                 Features.A_mean_rest = np.mean(A[init_point:end_point])
#                 Features.A_right_mean_rest = np.mean(A_r[init_point:end_point])
#                 Features.A_left_mean_rest = np.mean(A_l[init_point:end_point])
#                 Features.TB_mean_REST = np.mean(TB[init_point:end_point])
#                 Features.LL_mean_REST = np.mean(LL[init_point:end_point])
#                 Features.WM_mean_REST = np.mean(WM[init_point:end_point])

            datum.append(Subject_ID)
            datum.append(np.mean(A[init_point:end_point]))
            datum.append(np.mean(A_r[init_point:end_point]))
            datum.append(np.mean(A_l[init_point:end_point]))
            datum.append(np.mean(TB[init_point:end_point]))
            datum.append(np.mean(LL[init_point:end_point]))
            datum.append(np.mean(WM[init_point:end_point]))

            DataFrameResults = DataFrameResults.append(pd.Series(datum,index = columnsDF), ignore_index=True)
#         except:
#             pass

        
DataFrameResults

,Subject_ID,A_mean_rest,A_right_mean_rest,A_left_mean_rest,TB_mean_REST,LL_mean_REST,WM_mean_REST
0,NF12,700.299476,358.777830,341.521646,22.296298,56.326039,62.875664
1,NF13,532.446150,277.552557,254.893593,18.079827,43.033605,59.076115
2,NF14,1250.815619,635.059503,615.756116,28.950520,78.119554,86.469648
3,NF15,285.237679,167.414042,117.823638,12.000000,40.928644,47.547356
4,NF16,314.833196,162.899674,151.933522,12.066641,44.338532,52.192028
5,NF17,275.072214,138.336062,136.736152,11.021845,44.863639,50.037036
6,NF18,351.752114,189.365794,162.386320,14.124868,47.410034,50.250044
7,NF19,262.766575,139.629173,123.137402,10.261436,44.324350,51.300399
8,NF20,566.245359,288.647197,277.598163,21.484112,58.764004,52.740633
9,NF21,280.178907,124.523959,155.654948,10.376997,44.666146,54.037024


In [14]:
paths = [r'C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Healthy', r'C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Parkinsons']
tasks = ['REST', 'PA', 'BBP', 'BIGSMILE']

window_lenght_filter = 1


BIGSMILE_trials = pd.read_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\results_BIGSMILE.csv", index_col=0)
PA_trials = pd.read_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\results_PA.csv", index_col=0)
BBP_trials = pd.read_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\results_BBP_new.csv", index_col=0)



all_results = {}

all_results_data = {}

coord_sys= '2d'

for path in paths:
    Files = os.listdir(path)            
    if coor_sys == '3d':
        
        ext='Landmarks3D.csv'
    else:
        ext='landmarksFiltered.csv'
    Files = [i for i in Files if ext in i]
    for k,f in enumerate(Files):
        Files[k] = os.path.join(path,f)
        
        
        Subject_ID = f[0:4]  #get subject ID from file name
        #Features.Subject_status = f[0:2] #get disease status from file name
            
            
        this_task = None
        
        for t in tasks: 
            if t in f: this_task = t  #get task from file name
             
        DataFrame3dInfo = pd.read_csv(Files[k])
        Right_Corner_Coord, Left_Corner_Coord, Top_Lip_Coord, Bottom_Lip_Coord, Nose_Tip_Coord,_ = get_mouth_positions_3d(DataFrame3dInfo,coord_sys)  # get the four corners of the mouth :) 
        time_sec = DataFrame3dInfo['Time_Stamp (s)'].values[1:]
        
        TB_all = np.linalg.norm(Top_Lip_Coord - Bottom_Lip_Coord, axis =1)
        LL_all = np.linalg.norm(Nose_Tip_Coord - Bottom_Lip_Coord, axis =1)
        WM_all = np.linalg.norm(Right_Corner_Coord - Left_Corner_Coord, axis =1)
        Area_left_all = area_of_triangle(Left_Corner_Coord, Top_Lip_Coord, Bottom_Lip_Coord)
        Area_right_all = area_of_triangle(Right_Corner_Coord, Top_Lip_Coord, Bottom_Lip_Coord)
        #TB_all = signal.medfilt(TB_all, window_length_filter_all)
        
        fs = int(1/(time_sec[1]-time_sec[0]))
        nyq = 0.5 * fs
        highcut = 12.5
        order = 4
        high = highcut / nyq
        b, a = signal.butter(order,  high, btype='low')
        
        TB_all  = signal.filtfilt(b, a, TB_all)
        Area_left_all =  signal.filtfilt(b, a, Area_left_all)
        Area_right_all = signal.filtfilt(b, a, Area_right_all)
        Area = Area_left_all+Area_right_all
        LL_all  = signal.filtfilt(b, a, LL_all)
        WM_all  = signal.filtfilt(b, a, WM_all)
        
        
#         tt = TB_all[TB_all<0]
#         ww = WM_all[WM_all<0]
#         if len(tt)>0 or len(ww)>0:
#             print(Subject_ID, this_task)
    
        #compute excentricity of an ellipsis formed by the lips
        excentricity_all = compute_excentricity(WM_all, TB_all)
        
        #normalizeby the mean value obtained at rest before computing velocity and acceleration    
        TB_mean_rest = DataFrameResults.loc[DataFrameResults.Subject_ID == Subject_ID, 'TB_mean_REST'].values[0]
        WM_mean_rest = DataFrameResults.loc[DataFrameResults.Subject_ID == Subject_ID, 'WM_mean_REST'].values[0]
        LL_mean_rest = DataFrameResults.loc[DataFrameResults.Subject_ID == Subject_ID, 'LL_mean_REST'].values[0]
        A_mean_rest = DataFrameResults.loc[DataFrameResults.Subject_ID == Subject_ID, 'A_mean_rest'].values[0]
        
        TB_all = TB_all/TB_mean_rest
        LL_all = LL_all/LL_mean_rest
        WM_all = WM_all/WM_mean_rest
        Area = Area/A_mean_rest
        
        
        
        TB_all_vel = three_point_difference(TB_all,time_sec)
        LL_all_vel = three_point_difference(LL_all,time_sec)
        WM_all_vel = three_point_difference(WM_all,time_sec)
        
        TB_all_acc = three_point_difference(TB_all_vel,time_sec)
        LL_all_acc = three_point_difference(LL_all_vel,time_sec)
        WM_all_acc = three_point_difference(WM_all_vel,time_sec)
        if this_task is 'BIGSMILE':
            
            
            list_of_potential_trials = BIGSMILE_trials.loc[BIGSMILE_trials.Subject_ID==Subject_ID].values[0]
            
            list_of_results = np.zeros((18,0))
            normalized_trials = np.zeros((1000,0))
            for u, pp in enumerate(list_of_potential_trials):
                if isinstance(pp, str) and (':' in pp):
                    temp = pp.split(':')
                    init_point = int(temp[0])
                    end_point = int(temp[-1])
                    
                    #A_l = area_of_triangle(Left_Corner_Coord[init_point:end_point,:], Top_Lip_Coord[init_point:end_point,:], Bottom_Lip_Coord[init_point:end_point,:])
                    #A_r = area_of_triangle(Right_Corner_Coord[init_point:end_point,:], Top_Lip_Coord[init_point:end_point,:], Bottom_Lip_Coord[init_point:end_point,:])
                    A_l = Area_left_all[init_point:end_point]
                    A_r = Area_right_all[init_point:end_point]
                    A = Area[init_point:end_point]

                    all_results_data[(Subject_ID,this_task,'Area','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], A))

                    #TB = np.sqrt((Top_Lip_Coord[init_point:end_point,0]-Bottom_Lip_Coord[init_point:end_point,0])**2+(Top_Lip_Coord[init_point:end_point,1]-Bottom_Lip_Coord[init_point:end_point,1])**2+(Top_Lip_Coord[init_point:end_point,2]-Bottom_Lip_Coord[init_point:end_point,2])**2)
                    TB = TB_all[init_point:end_point]
                    LL = LL_all[init_point:end_point]
                    WM = WM_all[init_point:end_point]
                    TB_vel = TB_all_vel[init_point:end_point]
                    LL_vel = LL_all_vel[init_point:end_point]
                    WM_vel = WM_all_vel[init_point:end_point]
                    TB_acc = TB_all_acc[init_point:end_point]
                    LL_acc = LL_all_acc[init_point:end_point]
                    WM_acc = WM_all_acc[init_point:end_point]
                    excentricity= excentricity_all[init_point:end_point]
                    
                    #TB_vel_signal = signal.medfilt(np.gradient(signal.medfilt(TB,window_lenght_filter), time_sec[init_point:end_point]),window_lenght_filter)

                    all_results_data[(Subject_ID,this_task,'TB','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], TB))
                    all_results_data[(Subject_ID,this_task,'LL','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], LL))
                    all_results_data[(Subject_ID,this_task,'WM','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], WM))
                    
                    
                    

#                     temp_rest = [(time_sec[end_point]-time_sec[init_point]), 
#                                  (np.max(TB))/TB_mean_rest, 
#                                  (np.max(TB)-np.min(TB))/TB_mean_rest,
#                                  (np.max(WM))/WM_mean_rest,
#                                  (np.max(WM)-np.min(WM))/WM_mean_rest,
#                                  np.sum(np.abs(TB-TB_mean_rest)),
#                                  np.max(TB_vel),
#                                  np.min(TB_vel),
#                                  np.max(TB_acc),
#                                  np.min(TB_acc),
#                                  np.max(WM_vel),
#                                  np.min(WM_vel),
#                                  np.max(WM_acc),
#                                  np.min(WM_acc),
#                                  (np.mean(A))/A_mean_rest, 
#                                  ((np.max(A)-np.min(A)))/A_mean_rest,
#                                  concordance_correlation_coefficient(A_r, A_l),
#                                  np.max(excentricity)-np.min(excentricity)]
#                     list_of_results = np.column_stack((list_of_results, temp_rest)) 
                    temp_rest = [(time_sec[end_point]-time_sec[init_point]), 
                                 (np.max(TB)), 
                                 (np.max(TB)-np.min(TB)),
                                 (np.max(WM)),
                                 (np.max(WM)-np.min(WM)),
                                 np.sum(np.abs(TB)),
                                 np.max(TB_vel),
                                 np.min(TB_vel),
                                 np.max(TB_acc),
                                 np.min(TB_acc),
                                 np.max(WM_vel),
                                 np.min(WM_vel),
                                 np.max(WM_acc),
                                 np.min(WM_acc),
                                 (np.mean(A)), 
                                 ((np.max(A)-np.min(A))),
                                 concordance_correlation_coefficient(A_r, A_l),
                                 np.max(excentricity)-np.min(excentricity)]
                    list_of_results = np.column_stack((list_of_results, temp_rest)) 
                    
                    
                    #making the LL trajectory into a time-and-amplitude normalized vector
                    temp,_,_ = adjust_amplitude_and_time(LL, time_sec[init_point:end_point],normalize=True, des_n = 1000)
                    normalized_trials = np.column_stack((normalized_trials, temp)) 
             
            mean_normalized_trials = np.mean(normalized_trials,axis=1)
            feat =np.zeros((1,0))
            for trial in normalized_trials.T:
                temp = np.sqrt(np.mean((trial-mean_normalized_trials)**2))
                feat = np.column_stack((feat,temp))
                
            list_of_results = np.concatenate((list_of_results,feat))
            
            
#             Features :
#                 'Duration'
                    
#                 'Vertical ROM as a function of ROM_rest'
#                 'Horizontal ROM as a function of ROM_rest'
#                 'Cumulative path of lower lip movement normalized by LL position at rest'
#                 'Max LL velocity'
#                 'Min LL velocity'
#                 'Max LL acceleration'
#                 'Min LL acceleration'
#                 'Max WM velocity'
#                 'Min WM velocity'
#                 'Max WM acceleration'
#                 'Min WM acceleration'
#                 'Mean Area as a function of Area_rest'
#                 'Delta Area as a function of Area_rest'
#                 'CCC between Area left and Area right'
#                 'Range of mouth excentricity'
#                 'absolute difference of LL path with respect to mean trajectory'
                
                
            
            all_results[(Subject_ID,this_task,'Duration')] = list_of_results[0,:]
            all_results[(Subject_ID,this_task,'Max_Vertical')] = list_of_results[1,:]
            all_results[(Subject_ID,this_task,'Vertical_ROM')] = list_of_results[2,:]
            all_results[(Subject_ID,this_task,'Max_Horizontal')] = list_of_results[3,:]
            all_results[(Subject_ID,this_task,'Horizontal_ROM')] = list_of_results[4,:]
            all_results[(Subject_ID,this_task,'LL_Path')] = list_of_results[5,:]
            all_results[(Subject_ID,this_task,'Max_LL_vel')] = list_of_results[6,:]
            all_results[(Subject_ID,this_task,'Min_LL_vel')] = list_of_results[7,:]
            all_results[(Subject_ID,this_task,'Max_LL_acc')] = list_of_results[8,:]
            all_results[(Subject_ID,this_task,'Min_LL_acc')] = list_of_results[9,:]
            all_results[(Subject_ID,this_task,'Max_WM_vel')] = list_of_results[10,:]  
            all_results[(Subject_ID,this_task,'Min_WM_vel')] = list_of_results[11,:]
            all_results[(Subject_ID,this_task,'Max_WM_acc')] = list_of_results[12,:]
            all_results[(Subject_ID,this_task,'Min_WM_acc')] = list_of_results[13,:]
            all_results[(Subject_ID,this_task,'Mean_Area')] = list_of_results[14,:]
            all_results[(Subject_ID,this_task,'Delta_Area')] = list_of_results[15,:]
            all_results[(Subject_ID,this_task,'CCC_Area')] = list_of_results[16,:]
            all_results[(Subject_ID,this_task,'Range_excen')] = list_of_results[17,:]
            all_results[(Subject_ID,this_task,'LL_Path_RMSE')] = list_of_results[18,:]


        elif this_task is 'PA':
            
            list_of_potential_trials = PA_trials.loc[PA_trials.Subject_ID==Subject_ID].values[0]
            
            list_of_results = np.zeros((18,0))
            normalized_trials = np.zeros((1000,0))
            for u, pp in enumerate(list_of_potential_trials):
                if isinstance(pp, str) and (':' in pp):
                    temp = pp.split(':')
                    init_point = int(temp[0])
                    end_point = int(temp[-1])
                    
                    #A_l = area_of_triangle(Left_Corner_Coord[init_point:end_point,:], Top_Lip_Coord[init_point:end_point,:], Bottom_Lip_Coord[init_point:end_point,:])
                    #A_r = area_of_triangle(Right_Corner_Coord[init_point:end_point,:], Top_Lip_Coord[init_point:end_point,:], Bottom_Lip_Coord[init_point:end_point,:])
                    A_l = Area_left_all[init_point:end_point]
                    A_r = Area_right_all[init_point:end_point]
                    A = Area[init_point:end_point]

                    all_results_data[(Subject_ID,this_task,'Area','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], A))

                    #TB = np.sqrt((Top_Lip_Coord[init_point:end_point,0]-Bottom_Lip_Coord[init_point:end_point,0])**2+(Top_Lip_Coord[init_point:end_point,1]-Bottom_Lip_Coord[init_point:end_point,1])**2+(Top_Lip_Coord[init_point:end_point,2]-Bottom_Lip_Coord[init_point:end_point,2])**2)
                    TB = TB_all[init_point:end_point]
                    LL = LL_all[init_point:end_point]
                    WM = WM_all[init_point:end_point]
                    TB_vel = TB_all_vel[init_point:end_point]
                    LL_vel = LL_all_vel[init_point:end_point]
                    WM_vel = WM_all_vel[init_point:end_point]
                    TB_acc = TB_all_acc[init_point:end_point]
                    LL_acc = LL_all_acc[init_point:end_point]
                    WM_acc = WM_all_acc[init_point:end_point]
                    excentricity= excentricity_all[init_point:end_point]
                    
                    #TB_vel_signal = signal.medfilt(np.gradient(signal.medfilt(TB,window_lenght_filter), time_sec[init_point:end_point]),window_lenght_filter)

                    all_results_data[(Subject_ID,this_task,'TB','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], TB))
                    all_results_data[(Subject_ID,this_task,'LL','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], LL))
                    all_results_data[(Subject_ID,this_task,'WM','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], WM))


                    

                    #                     temp_rest = [(time_sec[end_point]-time_sec[init_point]), 
#                                  (np.max(TB))/TB_mean_rest, 
#                                  (np.max(TB)-np.min(TB))/TB_mean_rest,
#                                  (np.max(WM))/WM_mean_rest,
#                                  (np.max(WM)-np.min(WM))/WM_mean_rest,
#                                  np.sum(np.abs(TB-TB_mean_rest)),
#                                  np.max(TB_vel),
#                                  np.min(TB_vel),
#                                  np.max(TB_acc),
#                                  np.min(TB_acc),
#                                  np.max(WM_vel),
#                                  np.min(WM_vel),
#                                  np.max(WM_acc),
#                                  np.min(WM_acc),
#                                  (np.mean(A))/A_mean_rest, 
#                                  ((np.max(A)-np.min(A)))/A_mean_rest,
#                                  concordance_correlation_coefficient(A_r, A_l),
#                                  np.max(excentricity)-np.min(excentricity)]
#                     list_of_results = np.column_stack((list_of_results, temp_rest)) 
                    temp_rest = [(time_sec[end_point]-time_sec[init_point]), 
                                 (np.max(TB)), 
                                 (np.max(TB)-np.min(TB)),
                                 (np.max(WM)),
                                 (np.max(WM)-np.min(WM)),
                                 np.sum(np.abs(TB)),
                                 np.max(TB_vel),
                                 np.min(TB_vel),
                                 np.max(TB_acc),
                                 np.min(TB_acc),
                                 np.max(WM_vel),
                                 np.min(WM_vel),
                                 np.max(WM_acc),
                                 np.min(WM_acc),
                                 (np.mean(A)), 
                                 ((np.max(A)-np.min(A))),
                                 concordance_correlation_coefficient(A_r, A_l),
                                 np.max(excentricity)-np.min(excentricity)]
                    list_of_results = np.column_stack((list_of_results, temp_rest)) 
                    
                    
                    #making the LL trajectory into a time-and-amplitude normalized vector
                    temp,_,_ = adjust_amplitude_and_time(LL, time_sec[init_point:end_point],normalize=True, des_n = 1000)
                    normalized_trials = np.column_stack((normalized_trials, temp)) 
             
            mean_normalized_trials = np.mean(normalized_trials,axis=1)
            feat =np.zeros((1,0))
            for trial in normalized_trials.T:
                temp = np.sqrt(np.mean((trial-mean_normalized_trials)**2))
                feat = np.column_stack((feat,temp))
                
            list_of_results = np.concatenate((list_of_results,feat))
            
            
#             Features :
#                 'Duration'
                    
#                 'Vertical ROM as a function of ROM_rest'
#                 'Horizontal ROM as a function of ROM_rest'
#                 'Cumulative path of lower lip movement normalized by LL position at rest'
#                 'Max LL velocity'
#                 'Min LL velocity'
#                 'Max LL acceleration'
#                 'Min LL acceleration'
#                 'Max WM velocity'
#                 'Min WM velocity'
#                 'Max WM acceleration'
#                 'Min WM acceleration'
#                 'Mean Area as a function of Area_rest'
#                 'Delta Area as a function of Area_rest'
#                 'CCC between Area left and Area right'
#                 'Range of mouth excentricity'
#                 'absolute difference of LL path with respect to mean trajectory'
                
                
            
            all_results[(Subject_ID,this_task,'Duration')] = list_of_results[0,:]
            all_results[(Subject_ID,this_task,'Max_Vertical')] = list_of_results[1,:]
            all_results[(Subject_ID,this_task,'Vertical_ROM')] = list_of_results[2,:]
            all_results[(Subject_ID,this_task,'Max_Horizontal')] = list_of_results[3,:]
            all_results[(Subject_ID,this_task,'Horizontal_ROM')] = list_of_results[4,:]
            all_results[(Subject_ID,this_task,'LL_Path')] = list_of_results[5,:]
            all_results[(Subject_ID,this_task,'Max_LL_vel')] = list_of_results[6,:]
            all_results[(Subject_ID,this_task,'Min_LL_vel')] = list_of_results[7,:]
            all_results[(Subject_ID,this_task,'Max_LL_acc')] = list_of_results[8,:]
            all_results[(Subject_ID,this_task,'Min_LL_acc')] = list_of_results[9,:]
            all_results[(Subject_ID,this_task,'Max_WM_vel')] = list_of_results[10,:]  
            all_results[(Subject_ID,this_task,'Min_WM_vel')] = list_of_results[11,:]
            all_results[(Subject_ID,this_task,'Max_WM_acc')] = list_of_results[12,:]
            all_results[(Subject_ID,this_task,'Min_WM_acc')] = list_of_results[13,:]
            all_results[(Subject_ID,this_task,'Mean_Area')] = list_of_results[14,:]
            all_results[(Subject_ID,this_task,'Delta_Area')] = list_of_results[15,:]
            all_results[(Subject_ID,this_task,'CCC_Area')] = list_of_results[16,:]
            all_results[(Subject_ID,this_task,'Range_excen')] = list_of_results[17,:]
            all_results[(Subject_ID,this_task,'LL_Path_RMSE')] = list_of_results[18,:]
              
                    
        elif this_task is 'BBP':
                      
            list_of_potential_trials = BBP_trials.loc[BBP_trials.Subject_ID==Subject_ID].values[0]
            
            list_of_results = np.zeros((18,0))
            normalized_trials = np.zeros((1000,0))
            for u, pp in enumerate(list_of_potential_trials):
                if isinstance(pp, str) and (':' in pp):
                    temp = pp.split(':')
                    init_point = int(temp[0])
                    end_point = int(temp[-1])
                    
                    #A_l = area_of_triangle(Left_Corner_Coord[init_point:end_point,:], Top_Lip_Coord[init_point:end_point,:], Bottom_Lip_Coord[init_point:end_point,:])
                    #A_r = area_of_triangle(Right_Corner_Coord[init_point:end_point,:], Top_Lip_Coord[init_point:end_point,:], Bottom_Lip_Coord[init_point:end_point,:])
                    A_l = Area_left_all[init_point:end_point]
                    A_r = Area_right_all[init_point:end_point]
                    A = Area[init_point:end_point]

                    all_results_data[(Subject_ID,this_task,'Area','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], A))

                    #TB = np.sqrt((Top_Lip_Coord[init_point:end_point,0]-Bottom_Lip_Coord[init_point:end_point,0])**2+(Top_Lip_Coord[init_point:end_point,1]-Bottom_Lip_Coord[init_point:end_point,1])**2+(Top_Lip_Coord[init_point:end_point,2]-Bottom_Lip_Coord[init_point:end_point,2])**2)
                    TB = TB_all[init_point:end_point]
                    LL = LL_all[init_point:end_point]
                    WM = WM_all[init_point:end_point]
                    TB_vel = TB_all_vel[init_point:end_point]
                    LL_vel = LL_all_vel[init_point:end_point]
                    WM_vel = WM_all_vel[init_point:end_point]
                    TB_acc = TB_all_acc[init_point:end_point]
                    LL_acc = LL_all_acc[init_point:end_point]
                    WM_acc = WM_all_acc[init_point:end_point]
                    excentricity= excentricity_all[init_point:end_point]
                    
                    #TB_vel_signal = signal.medfilt(np.gradient(signal.medfilt(TB,window_lenght_filter), time_sec[init_point:end_point]),window_lenght_filter)

                    all_results_data[(Subject_ID,this_task,'TB','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], TB))
                    all_results_data[(Subject_ID,this_task,'LL','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], LL))
                    all_results_data[(Subject_ID,this_task,'WM','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], WM))
                    

                    #                     temp_rest = [(time_sec[end_point]-time_sec[init_point]), 
#                                  (np.max(TB))/TB_mean_rest, 
#                                  (np.max(TB)-np.min(TB))/TB_mean_rest,
#                                  (np.max(WM))/WM_mean_rest,
#                                  (np.max(WM)-np.min(WM))/WM_mean_rest,
#                                  np.sum(np.abs(TB-TB_mean_rest)),
#                                  np.max(TB_vel),
#                                  np.min(TB_vel),
#                                  np.max(TB_acc),
#                                  np.min(TB_acc),
#                                  np.max(WM_vel),
#                                  np.min(WM_vel),
#                                  np.max(WM_acc),
#                                  np.min(WM_acc),
#                                  (np.mean(A))/A_mean_rest, 
#                                  ((np.max(A)-np.min(A)))/A_mean_rest,
#                                  concordance_correlation_coefficient(A_r, A_l),
#                                  np.max(excentricity)-np.min(excentricity)]
#                     list_of_results = np.column_stack((list_of_results, temp_rest)) 
                    temp_rest = [(time_sec[end_point]-time_sec[init_point]), 
                                 (np.max(TB)), 
                                 (np.max(TB)-np.min(TB)),
                                 (np.max(WM)),
                                 (np.max(WM)-np.min(WM)),
                                 np.sum(np.abs(TB)),
                                 np.max(TB_vel),
                                 np.min(TB_vel),
                                 np.max(TB_acc),
                                 np.min(TB_acc),
                                 np.max(WM_vel),
                                 np.min(WM_vel),
                                 np.max(WM_acc),
                                 np.min(WM_acc),
                                 (np.mean(A)), 
                                 ((np.max(A)-np.min(A))),
                                 concordance_correlation_coefficient(A_r, A_l),
                                 np.max(excentricity)-np.min(excentricity)]
                    list_of_results = np.column_stack((list_of_results, temp_rest))  
                    
                    
                    #making the LL trajectory into a time-and-amplitude normalized vector
                    temp,_,_ = adjust_amplitude_and_time(LL, time_sec[init_point:end_point],normalize=True, des_n = 1000)
                    normalized_trials = np.column_stack((normalized_trials, temp)) 
             
            mean_normalized_trials = np.mean(normalized_trials,axis=1)
            feat =np.zeros((1,0))
            for trial in normalized_trials.T:
                temp = np.sqrt(np.mean((trial-mean_normalized_trials)**2))
                feat = np.column_stack((feat,temp))
                
            list_of_results = np.concatenate((list_of_results,feat))
            
            
#             Features :
#                 'Duration'
                    
#                 'Vertical ROM as a function of ROM_rest'
#                 'Horizontal ROM as a function of ROM_rest'
#                 'Cumulative path of lower lip movement normalized by LL position at rest'
#                 'Max LL velocity'
#                 'Min LL velocity'
#                 'Max LL acceleration'
#                 'Min LL acceleration'
#                 'Max WM velocity'
#                 'Min WM velocity'
#                 'Max WM acceleration'
#                 'Min WM acceleration'
#                 'Mean Area as a function of Area_rest'
#                 'Delta Area as a function of Area_rest'
#                 'CCC between Area left and Area right'
#                 'Range of mouth excentricity'
#                 'absolute difference of LL path with respect to mean trajectory'
                
                
            
            all_results[(Subject_ID,this_task,'Duration')] = list_of_results[0,:]
            all_results[(Subject_ID,this_task,'Max_Vertical')] = list_of_results[1,:]
            all_results[(Subject_ID,this_task,'Vertical_ROM')] = list_of_results[2,:]
            all_results[(Subject_ID,this_task,'Max_Horizontal')] = list_of_results[3,:]
            all_results[(Subject_ID,this_task,'Horizontal_ROM')] = list_of_results[4,:]
            all_results[(Subject_ID,this_task,'LL_Path')] = list_of_results[5,:]
            all_results[(Subject_ID,this_task,'Max_LL_vel')] = list_of_results[6,:]
            all_results[(Subject_ID,this_task,'Min_LL_vel')] = list_of_results[7,:]
            all_results[(Subject_ID,this_task,'Max_LL_acc')] = list_of_results[8,:]
            all_results[(Subject_ID,this_task,'Min_LL_acc')] = list_of_results[9,:]
            all_results[(Subject_ID,this_task,'Max_WM_vel')] = list_of_results[10,:]  
            all_results[(Subject_ID,this_task,'Min_WM_vel')] = list_of_results[11,:]
            all_results[(Subject_ID,this_task,'Max_WM_acc')] = list_of_results[12,:]
            all_results[(Subject_ID,this_task,'Min_WM_acc')] = list_of_results[13,:]
            all_results[(Subject_ID,this_task,'Mean_Area')] = list_of_results[14,:]
            all_results[(Subject_ID,this_task,'Delta_Area')] = list_of_results[15,:]
            all_results[(Subject_ID,this_task,'CCC_Area')] = list_of_results[16,:]
            all_results[(Subject_ID,this_task,'Range_excen')] = list_of_results[17,:]
            all_results[(Subject_ID,this_task,'LL_Path_RMSE')] = list_of_results[18,:]

            
df = pd.DataFrame.from_dict(all_results, orient='index')


#removed NF23 -> 
subjects = ['NF12', 'NF13', 'NF14', 'NF15', 'NF16', 'NF17', 'NF18', 'NF19', 'NF20', 'NF21', 'NF22', 'NF24', 'NF26', 'PD01', 'PD02', 'PD03', 'PD04', 'PD05', 'PD06', 'PD07', 'PD08']
tasks = ['BBP', 'PA', 'BIGSMILE']
task_features = ['Duration','Max_Vertical','Vertical_ROM','Max_Horizontal','Horizontal_ROM',
                 'LL_Path','Max_LL_vel','Min_LL_vel','Max_LL_acc', 'Min_LL_acc',
                 'Max_WM_vel','Min_WM_vel','Max_WM_acc', 'Min_WM_acc',
                 'Mean_Area','Delta_Area','CCC_Area','Range_excen','LL_Path_RMSE']
# features= [['Duration','max_TB', 'max_TB_vel', 'min_TB_vel','Mean_Area', 'Delta_Area','CCC_Area' ],
#                 ['Duration','max_TB', 'max_TB_vel', 'min_TB_vel'],
#                 ['Mean_Area', 'Delta_Area','CCC_Area']]

array_one = []
array_two = []
array_three = []
for c1 in subjects:
    for r,c2 in enumerate(tasks):
        for c3 in task_features:
            array_one.append(c1)
            array_two.append(c2)
            array_three.append(c3)
        
midx = pd.MultiIndex.from_arrays([array_one, array_two, array_three],  
          names =('Subject_ID', 'Task', 'Feature')) 
PD=pd.DataFrame(index = midx)
PD = pd.concat([PD,df], ignore_index=True, axis=1)
print('done')

done


In [12]:
PD

0           1           2   \
Subject_ID Task     Feature                                              
NF12       BBP      Duration          0.839508    0.819522    0.919466   
                    Max_Vertical      1.980249    1.673541    1.659643   
                    Vertical_ROM      1.020942    0.708580    0.692401   
                    Max_Horizontal    1.041560    1.041694    1.047151   
                    Horizontal_ROM    0.058599    0.107052    0.118291   
                    LL_Path          56.761040   52.321180   58.377130   
                    Max_LL_vel       17.218125   15.130168   13.827565   
                    Min_LL_vel      -15.421462  -13.973380  -11.640100   
                    Max_LL_acc      905.515781  943.251031  603.062043   
                    Min_LL_acc     -875.082746 -781.570483 -770.587825   
                    Max_WM_vel        1.744400    1.752252    2.494746   
                    Min_WM_vel       -1.730179   -1.895074   -2.871263   
                    Max_WM_acc      105.074931  126.092131  160.086380   
                    Min_WM_acc     -113.494730 -109.488710 -138.401961   
                    Mean_Area         1.343201    1.264319    1.236620   
                    Delta_Area        1.006081    0.690899    0.630007   
                    CCC_Area          0.807483    0.663555    0.420383   
                    Range_excen       0.238810    0.158311    0.140943   
                    LL_Path_RMSE      0.941395    1.108235    0.771256   
           PA       Duration          0.199881    0.159905    0.199881   
                    Max_Vertical      1.285565    1.146658    1.305181   
                    Vertical_ROM      0.508590    0.369682    0.536626   
                    Max_Horizontal    1.010594    1.006425    1.003321   
                    Horizontal_ROM    0.077942    0.005668    0.000302   
                    LL_Path          10.214244    7.795448   10.144546   
                    Max_LL_vel        9.094105    8.619032    8.362263   
                    Min_LL_vel       -9.809079   -5.988033  -13.482841   
                    Max_LL_acc      548.051812  299.853394  698.965520   
                    Min_LL_acc     -483.481481 -496.399511 -700.136477   
                    Max_WM_vel        2.319140    0.217117    0.030667   
...                                        ...         ...         ...   
PD08       PA       Max_LL_acc      487.167855  580.191557  750.697378   
                    Min_LL_acc     -281.840092 -477.341358 -633.731027   
                    Max_WM_vel        1.064146    1.359599    1.575018   
                    Min_WM_vel       -1.305802   -1.395274   -1.471478   
                    Max_WM_acc       47.840986   56.440833  113.179709   
                    Min_WM_acc      -46.612157  -76.855491  -94.205442   
                    Mean_Area         1.148280    1.127570    1.240217   
                    Delta_Area        0.940383    0.868897    1.052167   
                    CCC_Area          0.503957    0.609670    0.801516   
                    Range_excen       0.267726    0.238870    0.284406   
                    LL_Path_RMSE      0.208947    0.325292    0.122228   
           BIGSMILE Duration          2.066380    2.132536    2.065220   
                    Max_Vertical      1.516628    1.638601    1.519597   
                    Vertical_ROM      0.667292    0.765384    0.670895   
                    Max_Horizontal    1.192498    1.126152    1.040513   
                    Horizontal_ROM    0.220003    0.113459    0.079800   
                    LL_Path          81.185085   88.349779   79.771344   
                    Max_LL_vel        5.104712    6.846095    4.811459   
                    Min_LL_vel       -6.497848   -5.821331   -4.909251   
                    Max_LL_acc      493.112234  449.672455  313.955804   
                    Min_LL_acc     -340.898199 -402.032866 -366.037218   
                    Max_WM_vel        2.248082  

In [21]:
tasks = ['BBP', 'PA', 'BIGSMILE']
features= [['Duration','max_TB', 'max_TB_vel', 'min_TB_vel','Mean_Area', 'Delta_Area','CCC_Area' ],
                ['Duration','max_TB', 'max_TB_vel', 'min_TB_vel'],
                ['Mean_Area', 'Delta_Area','CCC_Area']]
task_features = ['Duration','Max_Vertical','Vertical_ROM','Max_Horizontal','Horizontal_ROM',
                 'LL_Path','Max_LL_vel','Min_LL_vel','Max_LL_acc', 'Min_LL_acc',
                 'Max_WM_vel','Min_WM_vel','Max_WM_acc', 'Min_WM_acc',
                 'Mean_Area','Delta_Area','CCC_Area','Range_excen','LL_Path_RMSE']

selected_features = ['Vertical_ROM','Max_LL_vel','Min_LL_vel','Max_LL_acc', 'Min_LL_acc',                
                 'Horizontal_ROM','Max_WM_vel','Min_WM_vel','Max_WM_acc', 'Min_WM_acc',
                 'Mean_Area','Delta_Area','CCC_Area','Range_excen']

array_two = []
array_three = []
SMD = []
M_h= []
STD_h = []
M_PD= []
STD_PD = []
for t,c2 in enumerate(tasks):
    for f,c3 in enumerate(selected_features):
        array_two.append(c2)
        array_three.append(c3)
        
        PD1 = PD.loc[subjects[0:13],c2,c3]
        PD2 = PD.loc[subjects[13:],c2,c3]
        tp1 = np.concatenate(PD1.values)
        tp1 = tp1[~np.isnan(tp1)]
        tp1 = tp1[np.nonzero(tp1)]
        tp2 = np.concatenate(PD2.values)
        tp2 = tp2[~np.isnan(tp2)]
        tp2 = tp2[np.nonzero(tp2)]
        
        #KW.append(stats.kruskal(tp1,tp2).pvalue)
        #ANOVA.append(stats.f_oneway(tp1,tp2).pvalue)
        
        #S = ((np.std(tp1)**2/len(tp1) + np.std(tp2)**2/len(tp2)) * (len(tp1)*len(tp2)/(len(tp1)+len(tp2))))
        S = np.sqrt(((len(tp1)-1)*np.var(tp1) + (len(tp2)-1)*np.var(tp2))/(len(tp1)+len(tp2)-2))
        SMD.append(np.round((np.mean(tp1) - np.mean(tp2))/S,2))
        
        M_h.append(np.mean(tp1))
        STD_h.append(np.std(tp1))
        M_PD.append(np.mean(tp2))
        STD_PD.append(np.std(tp2))
        
midx = pd.MultiIndex.from_arrays([array_two, array_three],  
          names =('Task', 'Feature')) 
PD_eval=pd.DataFrame(index = midx)
PD_eval['SMD'] = SMD
PD_eval['M-H'] = M_h
PD_eval['STD-H'] = STD_h
PD_eval['M-PD'] = M_PD
PD_eval['STD-PD'] = STD_PD
#PD_eval['ANOVA'] = ANOVA
#PD_eval['Kruskal–Wallis'] = KW
pd.options.display.float_format = '{:.4f}'.format
PD_eval

SMD        M-H    STD-H      M-PD   STD-PD
Task     Feature                                                      
BBP      Vertical_ROM    0.8000     1.2207   0.4280    0.9156   0.2901
         Max_LL_vel      0.8900    19.2545   6.2285   14.1413   4.7730
         Min_LL_vel     -0.6900   -20.0529   7.8517  -15.1753   5.6097
         Max_LL_acc      0.6800  1041.0667 430.8710  771.7121 329.8762
         Min_LL_acc     -0.7600 -1032.3205 383.7089 -761.9811 306.3393
         Horizontal_ROM  0.2800     0.1204   0.0340    0.1106   0.0354
         Max_WM_vel      0.1000     2.1817   0.6403    2.1172   0.7115
         Min_WM_vel     -0.3000    -2.2732   0.6636   -2.0668   0.7181
         Max_WM_acc      0.1300   135.8772  38.6859  130.3282  51.4040
         Min_WM_acc     -0.1200  -134.0817  38.3124 -129.0886  50.1317
         Mean_Area       0.4500     1.4638   0.2785    1.3484   0.2202
         Delta_Area      0.7800     1.1943   0.3952    0.9162   0.2850
         CCC_Area        0.6500     0.6339   0.2275    0.4739   0.2734
         Range_excen     0.1700     0.1772   0.1016    0.1596   0.1098
PA       Vertical_ROM    0.4000     0.8589   0.5879    0.6512   0.2918
         Max_LL_vel      0.4800    15.9603  10.4643   11.6101   4.7470
         Min_LL_vel     -0.4300   -17.5121  12.5837  -12.7531   5.6482
         Max_LL_acc      0.4100   856.2736 576.0171  651.2526 260.0411
         Min_LL_acc     -0.4400  -855.6046 594.4642 -628.9569 272.6764
         Horizontal_ROM  0.1000     0.0510   0.0402    0.0472   0.0370
         Max_WM_vel      0.1000     1.2860   0.9873    1.1883   0.9736
         Min_WM_vel     -0.0000    -1.1740   0.9619   -1.1698   0.9437
         Max_WM_acc     -0.0800    76.5641  57.3940   81.5270  73.3250
         Min_WM_acc      0.1000   -76.2354  57.5914  -82.2452  74.5692
         Mean_Area       0.0500     1.3571   0.3016    1.3440   0.1989
         Delta_Area      0.3700     0.7890   0.5312    0.6165   0.2702
         CCC_Area        0.8200     0.6468   0.2781    0.4219   0.2661
         Range_excen     0.2100     0.1282   0.1305    0.1036   0.0877
BIGSMILE Vertical_ROM    0.4800     0.9018   0.3789    0.7403   0.2605
         Max_LL_vel      0.6100    11.4045   4.1147    9.1577   2.9530
         Min_LL_vel     -0.2400   -10.3565   3.2228   -9.6235   2.8790
         Max_LL_acc      0.2200   725.1982 214.9477  671.6312 288.1058
         Min_LL_acc     -0.5500  -745.7958 226.8664 -622.5137 223.3751
         Horizontal_ROM  0.8400     0.2839   0.0653    0.2136   0.1074
         Max_WM_vel      0.2200     3.0280   0.8712    2.8229   1.0629
         Min_WM_vel     -0.6800    -3.3488   0.9053   -2.7303   0.9094
         Max_WM_acc      0.3400   203.5622  57.1369  184.0889  56.4434
         Min_WM_acc     -0.0900  -198.4662  61.5220 -192.6477  66.7663
         Mean_Area       0.3900     1.8789   0.3336    1.7352   0.4102
         Delta_Area      0.6100     1.3733   0.5094    1.0785   0.4433
         CCC_Area        1.2400     0.7889   0.1886    0.5138   0.2679
         Range_excen    -0.1600     0.0629   0.0388    0.0691   0.0388

In [20]:
%matplotlib qt
plt.rcParams.update({'font.size': 28})
plt.rc('text', usetex=False)
tasks = ['BBP', 'PA', 'BIGSMILE']
features= [['Duration','max_TB', 'max_TB_vel', 'min_TB_vel','Mean_Area', 'Delta_Area','CCC_Area' ],
           ['Duration','max_TB', 'max_TB_vel', 'min_TB_vel'],
           ['Mean_Area', 'Delta_Area','CCC_Area']]
task_features = ['Duration','Max_Vertical','Vertical_ROM','Max_Horizontal','Horizontal_ROM',
                 'LL_Path','Max_LL_vel','Min_LL_vel','Max_LL_acc', 'Min_LL_acc',
                 'Max_WM_vel','Min_WM_vel','Max_WM_acc', 'Min_WM_acc'
                 'Mean_Area','Delta_Area','CCC_Area','Range_excen','LL_Path_RMSE']

color_list = ['#DAD870', '#FFCD58', '#FF9636', '#FF5C4D', '#E80000', '#1E73BE', '#C850B0', '#660A60', '#010100', '#9BCCFD', '#0074DD', '#4120A9', '#B6D084', '#335120']
t = 2
f = 15

tp1 = np.empty((0,))
for k in range(13):
    if k == 66:
        continue
    else:
        PD1 = PD.loc[subjects[k],tasks[t],task_features[f]]
        temp = PD1.values
        temp = temp[~np.isnan(temp)]
        temp = temp[np.nonzero(temp)]

        rdnumbers = np.random.uniform(-0.075,0.075,len(temp))
        s =  rdnumbers+1
        plt.scatter(s,temp,c=color_list[k],alpha=0.5)

        tp1 = np.concatenate((temp, tp1))

tp2 = np.empty((0,))
for k in range(8):
    PD2 = PD.loc[subjects[k+13],tasks[t],task_features[f]]
    temp = PD2.values
    temp = temp[~np.isnan(temp)]
    temp = temp[np.nonzero(temp)]
    
    rdnumbers = np.random.uniform(-0.075,0.075,len(temp))
    s =  rdnumbers+2
    plt.scatter(s,temp,c=color_list[k],alpha=0.5)
    
    tp2 = np.concatenate((temp, tp2))
    
    
S = np.sqrt(((len(tp1)-1)*np.var(tp1) + (len(tp2)-1)*np.var(tp2))/(len(tp1)+len(tp2)-2))
SMD = (np.mean(tp1) - np.mean(tp2))/S

bp = plt.boxplot([tp1,tp2], notch=0, sym='', vert=1, whis=1.5, showfliers=False)
plt.setp(bp['boxes'], color='black',linewidth=4)
plt.setp(bp['whiskers'], color='black',linewidth=4)
plt.setp(bp['fliers'], color='red', marker='+',linewidth=4)
plt.setp(bp['medians'], color='red', linewidth=4)
plt.setp(bp['caps'], color='black',linewidth=4)
plt.xticks([1,2],['Healthy Control', 'Parkinsons Disease'])
plt.title(tasks[t]+' - '+task_features[f]+ ' -- '+str(np.round(SMD,2)))
plt.show()

In [6]:
paths = [r'C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Healthy', r'C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Parkinsons']
tasks = ['REST', 'PA', 'BBP', 'BIGSMILE']

window_lenght_filter = 1
window_length_filter_all = 3


BIGSMILE_trials = pd.read_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\results_BIGSMILE.csv", index_col=0)
PA_trials = pd.read_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\results_PA.csv", index_col=0)
BBP_trials = pd.read_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\results_BBP.csv", index_col=0)



all_results = {}
all_results_data = {}

for path in paths:
    Files = os.listdir(path)            
    ext='landmarksFiltered.csv'
    Files = [i for i in Files if ext in i]
    for k,f in enumerate(Files):
        Files[k] = os.path.join(path,f)
        
        Features = Variables_Placeholder()
        
        Features.Subject_ID = f[0:4]  #get subject ID from file name
        Features.Subject_status = f[0:2] #get disease status from file name
            
            
        this_task = None
        
        for t in tasks: 
            if t in f: this_task = t  #get task from file name
             
        DataFrame2dInfo = pd.read_csv(Files[k])
        Right_Corner_Coord, Left_Corner_Coord, Top_Lip_Coord, Bottom_Lip_Coord, norm_factor = get_mouth_positions_2d(DataFrame2dInfo)  # get the four corners of the mouth :) 
        time_sec = DataFrame2dInfo['Time_Stamp (s)'].values[1:]
        
        TB_all = np.linalg.norm(Top_Lip_Coord - Bottom_Lip_Coord, axis =1)
        Area_left_all = area_of_triangle(Left_Corner_Coord, Top_Lip_Coord, Bottom_Lip_Coord)
        Area_right_all = area_of_triangle(Right_Corner_Coord, Top_Lip_Coord, Bottom_Lip_Coord)
        
        fs = int(1/(time_sec[1]-time_sec[0]))
        nyq = 0.5 * fs
        highcut = 10
        order = 3
        high = highcut / nyq
        b, a = signal.butter(order,  high, btype='low')
        TB_all  = signal.filtfilt(b, a, TB_all)
        Area_left_all =  signal.filtfilt(b, a, Area_left_all)
        Area_right_all = signal.filtfilt(b, a, Area_right_all)

        if this_task is 'BIGSMILE':
            
            
            list_of_potential_trials = BIGSMILE_trials.loc[BIGSMILE_trials.Subject_ID==Features.Subject_ID].values[0]
            
            list_of_results = np.zeros((3,0))
            for u, pp in enumerate(list_of_potential_trials):
                try:
                    if (':' in pp):
                        temp = pp.split(':')
                        init_point = int(temp[0])
                        end_point = int(temp[-1])

                        #A_l = area_of_triangle(Left_Corner_Coord[init_point:end_point,:], Top_Lip_Coord[init_point:end_point,:], Bottom_Lip_Coord[init_point:end_point,:])
                        #A_r = area_of_triangle(Right_Corner_Coord[init_point:end_point,:], Top_Lip_Coord[init_point:end_point,:], Bottom_Lip_Coord[init_point:end_point,:])
                        A_l = Area_left_all[init_point:end_point]
                        A_r = Area_right_all[init_point:end_point]
                        A = A_l  + A_r



                        A_mean_rest = DataFrameResults.loc[DataFrameResults.Subject_ID == Features.Subject_ID, 'A_mean_rest'].values[0]
                        temp_rest = [np.mean((A-A_mean_rest)/A_mean_rest), ((np.max(A)-np.min(A))-A_mean_rest)/A_mean_rest,concordance_correlation_coefficient(A_r, A_l)]
                        list_of_results = np.column_stack((list_of_results, temp_rest))      
                        
                        all_results_data[(Features.Subject_ID,'BIGSMILE','Area','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], A))

                except:
                    pass
                    
            all_results[(Features.Subject_ID,'BIGSMILE','Mean_Area')] = list_of_results[0,:]
            all_results[(Features.Subject_ID,'BIGSMILE','Delta_Area')] = list_of_results[1,:]
            all_results[(Features.Subject_ID,'BIGSMILE','CCC_Area')] = list_of_results[2,:]    


        elif this_task is 'PA':
            
            list_of_potential_trials = PA_trials.loc[PA_trials.Subject_ID==Features.Subject_ID].values[0]
            
            list_of_results = np.zeros((4,0))
            for u, pp in enumerate(list_of_potential_trials):
                try:
                    if (':' in pp):
                        temp = pp.split(':')
                        init_point = int(temp[0])
                        end_point = int(temp[-1])
                        
                        TB_mean_rest = DataFrameResults.loc[DataFrameResults.Subject_ID == Features.Subject_ID, 'TB_mean_REST'].values[0]
                        
                        #TB = np.sqrt((Top_Lip_Coord[init_point:end_point,0]-Bottom_Lip_Coord[init_point:end_point,0])**2+(Top_Lip_Coord[init_point:end_point,1]-Bottom_Lip_Coord[init_point:end_point,1])**2)
                        TB = TB_all[init_point:end_point]
                        normalized_TB = TB/TB_mean_rest
                        TB_vel_signal = signal.medfilt(np.gradient(signal.medfilt(normalized_TB,window_lenght_filter), time_sec[init_point:end_point]),window_lenght_filter)

                        TB_mean_rest = DataFrameResults.loc[DataFrameResults.Subject_ID == Features.Subject_ID, 'TB_mean_REST'].values[0]

                        temp_rest = [(time_sec[end_point]-time_sec[init_point]), np.mean((np.max(TB)-TB_mean_rest)/TB_mean_rest), np.max(TB_vel_signal),np.min(TB_vel_signal)]
                        list_of_results = np.column_stack((list_of_results, temp_rest)) 
                        
                        all_results_data[(Features.Subject_ID,'PA','TB','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], TB)) 

                except:
                    pass

                    
            all_results[(Features.Subject_ID,'PA','Duration')] = list_of_results[0,:]
            all_results[(Features.Subject_ID,'PA','max_TB')] = list_of_results[1,:]
            all_results[(Features.Subject_ID,'PA','max_TB_vel')] = list_of_results[2,:]
            all_results[(Features.Subject_ID,'PA','min_TB_vel')] = list_of_results[3,:]
              
                    
        elif this_task is 'BBP':
            
            list_of_potential_trials = BBP_trials.loc[BBP_trials.Subject_ID==Features.Subject_ID].values[0]
            
            list_of_results = np.zeros((7,0))
            for u, pp in enumerate(list_of_potential_trials):
                try:
                    if (':' in pp):
                        temp = pp.split(':')
                        init_point = int(temp[0])
                        end_point = int(temp[-1])
                        
                        #A_l = area_of_triangle(Left_Corner_Coord[init_point:end_point,:], Top_Lip_Coord[init_point:end_point,:], Bottom_Lip_Coord[init_point:end_point,:])
                        #A_r = area_of_triangle(Right_Corner_Coord[init_point:end_point,:], Top_Lip_Coord[init_point:end_point,:], Bottom_Lip_Coord[init_point:end_point,:])
                        A_l = Area_left_all[init_point:end_point]
                        A_r = Area_right_all[init_point:end_point]
                        A = A_l  + A_r
                        
                        
                        TB_mean_rest = DataFrameResults.loc[DataFrameResults.Subject_ID == Features.Subject_ID, 'TB_mean_REST'].values[0]
                        
                        #TB = np.sqrt((Top_Lip_Coord[init_point:end_point,0]-Bottom_Lip_Coord[init_point:end_point,0])**2+(Top_Lip_Coord[init_point:end_point,1]-Bottom_Lip_Coord[init_point:end_point,1])**2)
                        TB = TB_all[init_point:end_point]
                        normalized_TB = TB/TB_mean_rest
                        TB_vel_signal = signal.medfilt(np.gradient(signal.medfilt(normalized_TB,window_lenght_filter), time_sec[init_point:end_point]),window_lenght_filter)

                        A_mean_rest = DataFrameResults.loc[DataFrameResults.Subject_ID == Features.Subject_ID, 'A_mean_rest'].values[0]

                        temp_rest = [(time_sec[end_point]-time_sec[init_point]), 
                                     np.mean((np.max(TB)-TB_mean_rest)/TB_mean_rest), 
                                     np.max(TB_vel_signal),
                                     np.min(TB_vel_signal),
                                     np.mean((A-A_mean_rest)/A_mean_rest), 
                                     ((np.max(A)-np.min(A))-A_mean_rest)/A_mean_rest,
                                     concordance_correlation_coefficient(A_r, A_l)]
                        list_of_results = np.column_stack((list_of_results, temp_rest))      
                        
                        all_results_data[(Features.Subject_ID,'BBP','Area','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], A))
                        all_results_data[(Features.Subject_ID,'BBP','TB','Trial'+str(u))] = np.column_stack((time_sec[init_point:end_point], TB))

                except:
                    pass

                    
            all_results[(Features.Subject_ID,'BBP','Duration')] = list_of_results[0,:]
            all_results[(Features.Subject_ID,'BBP','max_TB')] = list_of_results[1,:]
            all_results[(Features.Subject_ID,'BBP','max_TB_vel')] = list_of_results[2,:]
            all_results[(Features.Subject_ID,'BBP','min_TB_vel')] = list_of_results[3,:]
            all_results[(Features.Subject_ID,'BBP','Mean_Area')] = list_of_results[4,:]
            all_results[(Features.Subject_ID,'BBP','Delta_Area')] = list_of_results[5,:]
            all_results[(Features.Subject_ID,'BBP','CCC_Area')] = list_of_results[6,:]  

            

            
df = pd.DataFrame.from_dict(all_results, orient='index')

subjects = ['NF12', 'NF13', 'NF14', 'NF15', 'NF16', 'NF17', 'NF18', 'NF19', 'NF20', 'NF21', 'NF22', 'NF23', 'NF24', 'NF26', 'PD01', 'PD02', 'PD03', 'PD04', 'PD05', 'PD06', 'PD07', 'PD08']
tasks = ['BBP', 'PA', 'BIGSMILE']
features= [['Duration','max_TB', 'max_TB_vel', 'min_TB_vel','Mean_Area', 'Delta_Area','CCC_Area' ],
                ['Duration','max_TB', 'max_TB_vel', 'min_TB_vel'],
                ['Mean_Area', 'Delta_Area','CCC_Area']]

array_one = []
array_two = []
array_three = []
for c1 in subjects:
    for r,c2 in enumerate(tasks):
        for c3 in features[r]:
            array_one.append(c1)
            array_two.append(c2)
            array_three.append(c3)
        
columns = ['Subject_ID','Task','Feature']
midx = pd.MultiIndex.from_arrays([array_one, array_two, array_three],  
          names =('Subject_ID', 'Task', 'Feature')) 
PD=pd.DataFrame(index = midx)
PD = pd.concat([PD,df], ignore_index=True, axis=1)
print('done')

done


In [177]:
tasks = ['BBP', 'PA', 'BIGSMILE']
features= [['Duration','max_TB', 'max_TB_vel', 'min_TB_vel','Mean_Area', 'Delta_Area','CCC_Area' ],
                ['Duration','max_TB', 'max_TB_vel', 'min_TB_vel'],
                ['Mean_Area', 'Delta_Area','CCC_Area']]
array_two = []
array_three = []
SMD = []
KW =[]
ANOVA = []
for t,c2 in enumerate(tasks):
    for f,c3 in enumerate(features[t]):
        array_two.append(c2)
        array_three.append(c3)
        
        PD1 = PD.loc[subjects[0:13],tasks[t],features[t][f]]
        PD2 = PD.loc[subjects[14:],tasks[t],features[t][f]]
        tp1 = np.concatenate(PD1.values)
        tp1 = tp1[~np.isnan(tp1)]
        tp1 = tp1[np.nonzero(tp1)]
        tp2 = np.concatenate(PD2.values)
        tp2 = tp2[~np.isnan(tp2)]
        tp2 = tp2[np.nonzero(tp2)]
        
        KW.append(stats.kruskal(tp1,tp2).pvalue)
        ANOVA.append(stats.f_oneway(tp1,tp2).pvalue)

        S = ((np.std(tp1)**2/len(tp1) + np.std(tp2)**2/len(tp2)) * (len(tp1)*len(tp2)/(len(tp1)+len(tp2))))
        SMD.append(np.round(np.abs(np.mean(tp1) - np.mean(tp2))/S,2))
        
midx = pd.MultiIndex.from_arrays([array_two, array_three],  
          names =('Task', 'Feature')) 
PD_eval=pd.DataFrame(index = midx)
PD_eval['SMD'] = SMD
PD_eval['ANOVA'] = ANOVA
PD_eval['Kruskal–Wallis'] = KW
pd.options.display.float_format = '{:.4f}'.format
PD_eval

SMD  ANOVA  Kruskal–Wallis
Task     Feature                                 
BBP      Duration   0.6000 0.0447          0.0043
         max_TB     1.6200 0.0006          0.0089
         max_TB_vel 0.1600 0.0002          0.0003
         min_TB_vel 0.1300 0.0035          0.0178
         Mean_Area  2.5100 0.0041          0.1018
         Delta_Area 1.8700 0.0003          0.0012
         CCC_Area   1.6200 0.0048          0.0062
PA       Duration   4.4200 0.0123          0.0857
         max_TB     0.7900 0.0016          0.7121
         max_TB_vel 0.0900 0.0000          0.0170
         min_TB_vel 0.0700 0.0000          0.0275
BIGSMILE Mean_Area  1.0200 0.0402          0.0279
         Delta_Area 1.5100 0.0012          0.0017
         CCC_Area   5.7500 0.0000          0.0000

In [152]:
tasks = ['BBP', 'PA', 'BIGSMILE']
features= [['Duration','max_TB', 'max_TB_vel', 'min_TB_vel','Mean_Area', 'Delta_Area','CCC_Area' ],
           ['Duration','max_TB', 'max_TB_vel', 'min_TB_vel'],
           ['Mean_Area', 'Delta_Area','CCC_Area']]
t = 1
f = 2
PD1 = PD.loc[subjects[0:13],tasks[t],features[t][f]]
PD2 = PD.loc[subjects[14:],tasks[t],features[t][f]]
tp1 = np.concatenate(PD1.values)
tp1 = tp1[~np.isnan(tp1)]
tp1 = tp1[tp1>20]
tp2 = np.concatenate(PD2.values)
tp2 = tp2[~np.isnan(tp2)]
tp2 = tp2[tp2>20]

S = ((np.std(tp1)**2/len(tp1) + np.std(tp2)**2/len(tp2)) * (len(tp1)*len(tp2)/(len(tp1)+len(tp2))))
SMD = abs(np.mean(tp1) - np.mean(tp2))/S
%matplotlib qt
rdnumbers = np.random.uniform(-0.075,0.075,len(tp1))
s =  rdnumbers+1
plt.scatter(s,tp1,c='r',alpha=0.15)
rdnumbers = np.random.uniform(-0.075,0.075,len(tp2))
s =  rdnumbers+2
plt.scatter(s,tp2,c='r',alpha=0.15)
bp = plt.boxplot([tp1,tp2], notch=0, sym='', vert=1, whis=1.5, showfliers=False)
plt.setp(bp['boxes'], color='black',linewidth=2)
plt.setp(bp['whiskers'], color='black',linewidth=2)
plt.setp(bp['fliers'], color='red', marker='+',linewidth=2)
plt.setp(bp['medians'], color='blue', linewidth=2)
plt.setp(bp['caps'], color='black',linewidth=2)
plt.xticks([1,2],['Healthy Control', 'Parkinsons Disease'])
plt.title(tasks[t]+' - '+features[t][f]+ ' -- '+str(np.round(SMD,2)))
plt.show()

In [165]:
((np.std(tp1)**2/len(tp1) + np.std(tp2)**2/len(tp2)) * (len(tp1)*len(tp2)/(len(tp1)+len(tp2))))

5861.7861267145545

In [137]:
sg = all_results_data[('NF12','PA','TB','Trial2')]
segment = sg[:,1]
t_v = sg[:,0]
window_lenght_filter =1
#TB = np.sqrt((Top_Lip_Coord[init_point:end_point,0]-Bottom_Lip_Coord[init_point:end_point,0])**2+(Top_Lip_Coord[init_point:end_point,1]-Bottom_Lip_Coord[init_point:end_point,1])**2+(Top_Lip_Coord[init_point:end_point,2]-Bottom_Lip_Coord[init_point:end_point,2])**2)
vel_segment = signal.medfilt(np.gradient(signal.medfilt(segment,window_lenght_filter), t_v),window_lenght_filter)
#print(this_task,np.max(TB_vel_signal),np.min(TB_vel_signal))
plt.plot(t_v,vel_segment,'o--')


In [122]:
TB = np.sqrt((Top_Lip_Coord[:,0]-Bottom_Lip_Coord[:,0])**2+(Top_Lip_Coord[:,1]-Bottom_Lip_Coord[:,1])**2)                   
plt.plot(np.linalg.norm(Top_Lip_Coord - Bottom_Lip_Coord, axis=1))
plt.plot(TB)

In [176]:
tasks = ['BBP', 'PA', 'BIGSMILE']
features= [['Duration','max_TB', 'max_TB_vel', 'min_TB_vel','Mean_Area', 'Delta_Area','CCC_Area' ],
           ['Duration','max_TB', 'max_TB_vel', 'min_TB_vel'],
           ['Mean_Area', 'Delta_Area','CCC_Area']]
t = 0
f = 0
PD1 = PD.loc[subjects[0:13],tasks[t],features[t][f]]
PD2 = PD.loc[subjects[14:],tasks[t],features[t][f]]
tp11 = np.concatenate(PD1.values)
tp11 = tp11[~np.isnan(tp11)]
tp21 = np.concatenate(PD2.values)
tp21 = tp21[~np.isnan(tp21)]

t = 0
f = 6
PD1 = PD.loc[subjects[0:13],tasks[t],features[t][f]]
PD2 = PD.loc[subjects[14:],tasks[t],features[t][f]]
tp12 = np.concatenate(PD1.values)
tp12 = tp12[~np.isnan(tp12)]
tp22 = np.concatenate(PD2.values)
tp22 = tp22[~np.isnan(tp22)]
plt.scatter(tp11,tp12)
plt.scatter(tp21,tp22)

In [52]:
subjects = ['NF12', 'NF13', 'NF14', 'NF15', 'NF16', 'NF17', 'NF18', 'NF19', 'NF20', 'NF21', 'NF22', 'NF23', 'NF24', 'NF26', 'PD01', 'PD02', 'PD03', 'PD04', 'PD05', 'PD06', 'PD07', 'PD08']
tasks = ['BBP', 'PA', 'BIGSMILE']
features= [['Duration','max_TB', 'max_TB_vel', 'min_TB_vel','Mean_Area', 'Delta_Area','CCC_Area' ],
                ['Duration','max_TB', 'max_TB_vel', 'min_TB_vel'],
                ['Mean_Area', 'Delta_Area','CCC_Area']]


df_cols_BBP= ['Subject_ID','Duration','max_TB', 'max_TB_vel', 'min_TB_vel','Mean_Area', 'Delta_Area','CCC_Area','Label']
DF_BBP= pd.DataFrame(columns = df_cols_BBP)
df_cols_PA= ['Subject_ID','Duration','max_TB', 'max_TB_vel', 'min_TB_vel','Label']
DF_PA= pd.DataFrame(columns = df_cols_PA)
df_cols_BIGSMILE= ['Subject_ID','Mean_Area', 'Delta_Area','CCC_Area','Label']
DF_BIGSMILE= pd.DataFrame(columns = df_cols_BIGSMILE)


for t,c2 in enumerate(tasks):
    for s in subjects:
        to_carry = np.empty(0)
        
        if c2 == 'BBP':
            PD1 = PD.loc[s, c2].values[:]
            n_trials = PD1[0][~np.isnan(PD1[0])].shape[0]
            n_features = PD1.shape[0]
            to_carry = np.broadcast_to(np.array(s),(n_trials,1))
            for m in PD1:
                m1 = m[~np.isnan(m)]
                to_carry = np.column_stack((to_carry,m1))
            
            if 'NF' in s:
                to_carry=np.column_stack((to_carry,np.zeros((to_carry.shape[0],1))))
            else:
                to_carry=np.column_stack((to_carry,np.ones((to_carry.shape[0],1))))
            
            for m in to_carry:
                DF_BBP = DF_BBP.append(pd.Series(m,index = df_cols_BBP), ignore_index = True)
            
        if c2 == 'PA':
            PD1 = PD.loc[s, c2].values[:]
            n_trials = PD1[0][~np.isnan(PD1[0])].shape[0]
            n_features = PD1.shape[0]
            to_carry = np.broadcast_to(np.array(s),(n_trials,1))
            for m in PD1:
                m1 = m[~np.isnan(m)]
                to_carry = np.column_stack((to_carry,m1))
            
            if 'NF' in s:
                to_carry=np.column_stack((to_carry,np.zeros((to_carry.shape[0],1))))
            else:
                to_carry=np.column_stack((to_carry,np.ones((to_carry.shape[0],1))))
            
            for m in to_carry:
                DF_PA = DF_PA.append(pd.Series(m,index = df_cols_PA), ignore_index = True)
            
            
        if c2 == 'BIGSMILE':
            PD1 = PD.loc[s, c2].values[:]
            n_trials = PD1[0][~np.isnan(PD1[0])].shape[0]
            n_features = PD1.shape[0]
            to_carry = np.broadcast_to(np.array(s),(n_trials,1))
            for m in PD1:
                m1 = m[~np.isnan(m)]
                to_carry = np.column_stack((to_carry,m1))
            
            if 'NF' in s:
                to_carry=np.column_stack((to_carry,np.zeros((to_carry.shape[0],1))))
            else:
                to_carry=np.column_stack((to_carry,np.ones((to_carry.shape[0],1))))
            
            for m in to_carry:
                DF_BIGSMILE = DF_BIGSMILE.append(pd.Series(m,index = df_cols_BIGSMILE), ignore_index = True)
        

DF_BBP.to_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Features2d_BBP.csv")
DF_PA.to_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Features2d_PA.csv")
DF_BIGSMILE.to_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Features2d_BIGSMILE.csv")

C:\Users\GuarinD\.conda\envs\pytorch_deploy\lib\site-packages\ipykernel_launcher.py:21: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\GuarinD\.conda\envs\pytorch_deploy\lib\site-packages\ipykernel_launcher.py:38: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\GuarinD\.conda\envs\pytorch_deploy\lib\site-packages\ipykernel_launcher.py:56: PerformanceWarning: indexing past lexsort depth may impact performance.


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

In [20]:
X = DF_BBP_2d.iloc[:, 1:7].values
y = DF_BBP_2d.iloc[:, 7].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
regressor = RandomForestClassifier()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
print(regressor.feature_importances_)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[0.18563222 0.19246964 0.09372514 0.19105248 0.17840006 0.15872046]
[[21  5]
 [ 6  9]]
              precision    recall  f1-score   support

         0.0       0.78      0.81      0.79        26
         1.0       0.64      0.60      0.62        15

    accuracy                           0.73        41
   macro avg       0.71      0.70      0.71        41
weighted avg       0.73      0.73      0.73        41

0.7317073170731707


In [31]:
from sklearn.model_selection import StratifiedShuffleSplit
 
    

In [50]:
X = DF_BBP_2d.iloc[:, 1:7].values
y = DF_BBP_2d.iloc[:, 7].values
idx = range(0,len(X))
ss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
regressor = RandomForestClassifier()
acc = []
feat_importance = []
for train_index, test_index in ss.split(X,y):
    X_train, y_train, X_test, y_test = X[train_index], y[train_index], X[test_index],y[test_index]
    regressor.fit(X_train, y_train.astype(int))
    y_pred = regressor.predict(X_test)
    acc.append(accuracy_score(y_test, y_pred))
    feat_importance.append(regressor.feature_importances_)

print(np.mean(acc), np.std(acc))
print(np.mean(np.array(feat_importance),axis=0))

0.8195121951219513 0.050222586053595114
[0.15561512 0.14131614 0.11628732 0.21586682 0.1666341  0.20428049]


In [37]:
test_index

array([ 88, 174,  77,  81,  37, 169, 147, 117,   9,  65, 183, 121, 128,
        67, 144,  21, 123,  46, 114, 143,  12,  58,  39, 155,  87,  25,
        47, 173, 153,  70,  83, 102, 120,  72, 103, 160, 176, 134,  36,
       191, 135], dtype=int64)